### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [ ]:
!wget https://dataset-ppm.s3.amazonaws.com/iris.csv

--2021-03-20 11:55:45--  https://dataset-ppm.s3.amazonaws.com/iris.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.217.128.201
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.217.128.201|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4609 (4.5K) [text/csv]
Saving to: ‘iris.csv.1’

iris.csv.1          100%[===================>]   4.50K  --.-KB/s    in 0s      

2021-03-20 11:55:45 (123 MB/s) - ‘iris.csv.1’ saved [4609/4609]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [ ]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [ ]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size = 0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [ ]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya **label_latih** dan **label_uji**

In [ ]:
label_latih = data_latih.pop('species')
label_uji = data_uji.pop('species')

In [ ]:
#Cek data latih dan data uji setelah melalui proses operasi pop()
print("Data latih\n")
print(data_latih.head())
print("")
print("Data uji\n")
print(data_uji.head())


Data latih

     sepal_length  sepal_width  petal_length  petal_width
62            6.0          2.2           4.0          1.0
77            6.7          3.0           5.0          1.7
41            4.5          2.3           1.3          0.3
121           5.6          2.8           4.9          2.0
15            5.7          4.4           1.5          0.4

Data uji

     sepal_length  sepal_width  petal_length  petal_width
93            5.0          2.3           3.3          1.0
120           6.9          3.2           5.7          2.3
59            5.2          2.7           3.9          1.4
67            5.8          2.7           4.1          1.0
137           6.4          3.1           5.5          1.8


## 3) Menghitung jarak euclidean

Tahapan awal dari algoritma KNN adalah perhitungan jarak. Salah satu metode perhitungan jarak yang bisa digunakan adalah jarak Euclidean. Buatlah fungsi bernama **jarakEu** yang berfungsi menghitung jarak euclidean dari dua buah vektor (tanpa kelas)

In [ ]:
def jarakEu(data1,data2):
  jarak = np.square(data1-data2)
  jarak = np.sum(jarak)
  return np.sqrt(jarak)

Uji fungsi **jarakEu** untuk menghitung jarak antara data latih pertama dengan data uji pertama.

In [ ]:
jarak = jarakEu(data_latih.iloc[0],data_uji.iloc[0])
print(jarak)

1.2247448713915892


## 4) Algoritma KNN

Implementasikan algoritma KNN dengan tahapan-tahapan berikut :
1. Hitung jarak antara data uji dengan setiap data latih.
2. Cari *k* data latih dengan jarak terkecil.
3. Tentukan kelas dari *k* data latih tersebut.
4. Kelas data uji ditentukan dari mayoritas kelas *k* data latih.

Buatlah fungsi dengan nama **knn** yang menerima input berupa *k*, sekumpulan data latih dan labelnya, serta sebuah data uji

In [ ]:
from collections import Counter

def knn(k,datalatih,labellatih,datauji):
  jarak = np.array([jarakEu(datalatih.iloc[x],datauji) for x in range (datalatih.shape[0])]) 
  indeks_k_minimum = jarak.argsort()[:k]
  k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy()
  counter = Counter(k_kelas)
  kelas_uji = counter.most_common(1)[0][0]
  return kelas_uji

Lakukan pengujian fungsi **knn** untuk menentukan kelas dari data uji pertama

In [ ]:
result = knn(3,data_latih,label_latih,data_uji.iloc[0])
print(result)

Iris-versicolor


## 5) KNN pada Scikit-learn

Library scikit-learn memiliki algoritma KNN yang siap untuk digunakan. Sebagai pengenalan, Anda akan melakukan klasifikasi data yang sama menggunakan KNN yang tersedia pada library scikit-learn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 3)
KNN.fit(data_latih, label_latih)
kelas = KNN.predict([data_uji.iloc[0]])
print(kelas)

['Iris-versicolor']


## TUGAS
1. Buatlah fungsi bernama **jarakCity** yang berfungsi menghitung jarak City Block antara dua vektor.
2. Buatlah sebuah fungsi bernama **knn_multi** yang dapat menentukan kelas dari **semua** data uji (Fungsi **knn** yang sudah Anda buat hanya dapat menentukan kelas dari sebuah data uji. Anda dapat memanfaatkan fungsi **knn** yang telah ada
3. Buatlah fungsi bernama **cek_hasil** yang berfungsi membandingkan hasil klasifikasi dengan label sebenarnya. Fungsi ini menerima input 2 buah list, yaitu list label hasil klasifikasi dan list label data uji yang sebenarnya. Output dari fungsi ini adalah integer, yang menunjukkan berapa banyak label hasil klasifikasi yang sama dengan label data uji sebenarnya.
4. Jalankan fungsi **knn_multi** dengan nilai k=3, k=7,k=9 dan k=13. Amati hasil dari fungsi **cek_hasil** dengan nilai k tersebut. Apa kesimpulan Anda?

In [ ]:
#jawaban soal 1
#Manhattan / City Block Distance
def jarakCity(vektor1, vektor2):
  distance = np.abs(vektor1 - vektor2)
  distance = np.sum(distance)
  return distance

In [ ]:
JarakManhattan = jarakCity(data_latih.iloc[0],data_uji.iloc[0])
print(JarakManhattan)

1.7999999999999998


In [ ]:
#jawaban soal 2
def knn_multi(k,data_latih,label_latih,data_uji):
  daftar_class = []
  
  for x in range(data_uji.shape[0]):
    result_class = knn(k,data_latih,label_latih,data_uji.iloc[x])
    daftar_class.append(result_class)

  return daftar_class


In [ ]:
#Melihat hasil, contoh dengan k = 3
result_KNNMulti = knn_multi(3,data_latih,label_latih,data_uji)
result_KNNMulti

['Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-virginica']

In [ ]:
#Mengecek data uji yang di dapatkan setelah di split
data_uji

In [ ]:
#Untuk looping nya.
#for x in range(data_uji.shape[0]):
#  print(data_uji.iloc[x])

In [ ]:
#jawaban soal 3
def cek_hasil(result, label_uji):
  Data_sama = 0
  for x in range(0, len(label_uji)):
    if(label_uji.iloc[x] == result[x]):
      Data_sama += 1
  
  return Data_sama

In [ ]:
#cek hasil
print("Jumlah data yang sama :", cek_hasil(result_KNNMulti, label_uji))
print("")
print("Data yang di compare sebanyak :", label_uji.shape[0])

Jumlah data yang sama : 28

Data yang di compare sebanyak : 30


In [ ]:
#jawaban soal 4
#Run fungsi knn_multi untuk k = 3, 7, 9 dan 13
result_k3 = knn_multi(3,data_latih,label_latih,data_uji)
result_k7 = knn_multi(7,data_latih,label_latih,data_uji)
result_k9 = knn_multi(9,data_latih,label_latih,data_uji)
result_k13 = knn_multi(13,data_latih,label_latih,data_uji)

In [ ]:
def printCompare(result, label_uji):
  print("Jumlah data yang sama dengan Data uji : ", cek_hasil(result, label_uji))
  print("Jumlah data yang dibandingkan : ", label_uji.shape[0])

In [ ]:
print("K = 3")
printCompare(result_k3, label_uji)
print("")
print("K = 7")
printCompare(result_k7, label_uji)
print("")
print("K = 9")
printCompare(result_k9, label_uji)
print("")
print("K = 13")
printCompare(result_k13, label_uji)

K = 3
Jumlah data yang sama dengan Data uji :  28
Jumlah data yang dibandingkan :  30

K = 7
Jumlah data yang sama dengan Data uji :  27
Jumlah data yang dibandingkan :  30

K = 9
Jumlah data yang sama dengan Data uji :  30
Jumlah data yang dibandingkan :  30

K = 13
Jumlah data yang sama dengan Data uji :  29
Jumlah data yang dibandingkan :  30


Penjelasan jawaban soal nomor 4

KNN / *K-nearest neighbors* merupakan algoritma yang digunakan untuk melakukan klasifikasi suatu data uji dengan data latih. K dalam KNN merupakan banyaknya tetangga terdekat / *nearest neighbors*.

1. Berdasarkan percobaan - percobaan sebelumnya, Hasil akhir dari data yang kita buat dengan data uji yang asli relatif memiliki kemiripan yang tinggi.

2. K pada KNN menentukan jumlah kelas yang akan di jadikan perbandingan untuk nantinya dilakukan majority voting, berdasarkan voting terbanyak maka itu merupakan class untuk data latih tersebut. 

3. Kelemahan yang ada pada Algoritma KNN ini adalah
  1. Jika K terlalu kecil, maka akan sangat sensitif terhadap noise/outlier
  2. Jika K terlalu besar, maka penentuan class nya dapat dipengaruhi oleh data dari class yang lainnya.
  
  Hal ini dapat dilihat pada k = 3 dan k = 13, dimana yang terkecil dan terbesar dalam contoh praktikum ini. Setelah dilakukan berulang kali percobaan maka K terkecil dan K terbesar hampir memiliki data yang sama. Namun semisal K terlalu kecil misal k = 1, maka data belum tentu akurat karena dapat dipengaruhi oleh noise data. Biasanya semakin besar nilai k maka semakin baik dalam penentuan klasifikasi datanya. Namun, besarnya K tergantung dengan jumlah dataset yang disediakan. Pada kasus ini kita disediakan 30 dataset maka tidak layaknya kita men-set K = 30.

  Kesimpulan dari K ini, banyaknya K tidak dapat ditentukan dengan pasti karena terlalu bergantung dengan beberapa kondisi maka dari itu perlu dilakukan beberapa percobaan dengan K yang berbeda - beda. Kemudian untuk nilai K harus bernilai ganjil, Karena jika bernilai genap maka saat melakukan majority voting bisa terjadi adanya jumlah vote yang sama yang kemudian tidak dapat menghasilkan majority.

4. KNN juga memiliki tingkat kompleksitas yang tinggi karena dalam melakukan perhitungan jarak dilakukan terhadap setiap data latih. Maka jika semakin banyak jumlah data latih maka semakin tinggi komputasinya.

5. Pada praktikum ini, Kita disediakan 150 data. Namun dengan menggunakan data split, kita mengambil hanya 20% nya saja yaitu 30 data secara random, maka dari itu saat melakukan factory reset pada runtime dan me - run ulang akan menghasilkan hasil yang berbeda lagi. Hal ini juga menujukan bahwa jika kita memilih nilai K yang besar dan karena datanya random bisa jadi bahwa sebuah data seharusnya masuk kedalam class A namun justru masuk kedalam class B. Karena jumlah data pada class B lebih banyak dari A namun pada class B juga memiliki kemiripan pada A, jadinya data masuk ke class yang salah. 
